<h1> A Data Science Project About Nothing </h1>

<h3> <i>Seinfeld</i>, my favourite sitcom and one commonly referred to as 'a show about nothing', will be the subject of this trial-and-error-driven project. Join me on my journey! </h3>

<h4> Scraping IMDb for <i>Seinfeld</i> data, including ratings. </h4>

I originally tried to do this using the 'old' method, but IMDb has changed its website layout to be dynamic, which meant I needed to use Selenium to automate the browser interaction.

The biggest hurdle was figuring out the correct format for the CSS Selectors that corresponded with the info I wanted to scrape.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import csv  


chrome_options = Options()
chrome_options.add_argument("--headless") 
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")


service = Service('/usr/local/bin/chromedriver')  


driver = webdriver.Chrome(service=service, options=chrome_options)
wait = WebDriverWait(driver, 10)

# Placing the function within a try/finally block

try:

    def get_imdb_ratings(season):
        url = f'https://www.imdb.com/title/tt0098904/episodes?season={season}'
        driver.get(url)
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.sc-9115db22-1.egUMEz')))

        episodes = driver.find_elements(By.CSS_SELECTOR, '.sc-9115db22-1.egUMEz')
        episode_data = []

        for episode in episodes:
            title = episode.find_element(By.CSS_SELECTOR, '.ipc-title__text').text
            airdate = episode.find_element(By.CSS_SELECTOR, '.sc-9115db22-10.jAfkDE').text.strip()
            try:
                # Corrected CSS selector by chaining class names
                rating = episode.find_element(By.CSS_SELECTOR, '.ipc-rating-star.ipc-rating-star--base.ipc-rating-star--imdb.ratingGroup--imdb-rating').text
            except Exception as e:
                rating = 'N/A'

            episode_data.append({'title': title, 'airdate': airdate, 'rating': rating})

        return episode_data

    # Gather data for all seasons
    all_seasons_data = []
    for season in range(1, 10):  # Seinfeld seasons 1 to 9
        season_data = get_imdb_ratings(season)
        for episode_info in season_data:
            episode_info['season'] = season  # Add season number to each episode
            all_seasons_data.append(episode_info)

    with open('seinfeld_ratings.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Title', 'Airdate', 'Rating', 'Season'])

        
        for episode in all_seasons_data:
            writer.writerow([episode['title'], episode['airdate'], episode['rating'], episode['season']])

    # Convert the list of dictionaries to a dataframe
    df = pd.DataFrame(all_seasons_data)

    df.to_csv('/Users/sara/seinfeld/seinfeld_ratings.csv', index=False)

finally:
            driver.quit()


Next, I wanted to split the column that shows the episode number and season into two separate columns.
For this, I needed to use a regex expression that would capture the S for season and E for episode.

In [ ]:
# Split the 'episode_info' column where the format follows a 'S7.E8 ∙ Episode Title' pattern
df[['season_episode', 'episode_title']] = df['title'].str.split(' ∙ ', expand=True)

df[['season', 'episode']] = df['season_episode'].str.extract(r'S(\d+)\.E(\d+)', expand=True)

# Convert 'season' and 'episode' to numeric types
df['season'] = pd.to_numeric(df['season'], errors='coerce')
df['episode'] = pd.to_numeric(df['episode'], errors='coerce')

print(df.head())

                         title            airdate             rating  season  \
0  S1.E1 ∙ Good News, Bad News   Wed, Oct 6, 1993  7.3\n/10\n (6.4K)       1   
1         S1.E2 ∙ The Stakeout  Thu, May 31, 1990  7.5\n/10\n (5.5K)       1   
2          S1.E3 ∙ The Robbery   Thu, Jun 7, 1990    7.5\n/10\n (5K)       1   
3       S1.E4 ∙ Male Unbonding  Thu, Jun 14, 1990  7.3\n/10\n (4.9K)       1   
4        S1.E5 ∙ The Stock Tip  Thu, Jun 21, 1990  7.4\n/10\n (4.8K)       1   

  season_episode        episode_title  episode  
0          S1.E1  Good News, Bad News        1  
1          S1.E2         The Stakeout        2  
2          S1.E3          The Robbery        3  
3          S1.E4       Male Unbonding        4  
4          S1.E5        The Stock Tip        5  


In [ ]:
# Splitting the 'episode_info' column
df[['season_episode', 'episode_title']] = df['title'].str.split(' ∙ ', expand=True)

df[['season', 'episode']] = df['season_episode'].str.extract(r'S(\d+)\.E(\d+)', expand=True)

# Converting 'season' and 'episode' to numeric types
df['season'] = pd.to_numeric(df['season'], errors='coerce')
df['episode'] = pd.to_numeric(df['episode'], errors='coerce')

print(df.head())

                         title            airdate             rating  season  \
0  S1.E1 ∙ Good News, Bad News   Wed, Oct 6, 1993  7.3\n/10\n (6.4K)       1   
1         S1.E2 ∙ The Stakeout  Thu, May 31, 1990  7.5\n/10\n (5.5K)       1   
2          S1.E3 ∙ The Robbery   Thu, Jun 7, 1990    7.5\n/10\n (5K)       1   
3       S1.E4 ∙ Male Unbonding  Thu, Jun 14, 1990  7.3\n/10\n (4.9K)       1   
4        S1.E5 ∙ The Stock Tip  Thu, Jun 21, 1990  7.4\n/10\n (4.8K)       1   

  season_episode        episode_title  episode  
0          S1.E1  Good News, Bad News        1  
1          S1.E2         The Stakeout        2  
2          S1.E3          The Robbery        3  
3          S1.E4       Male Unbonding        4  
4          S1.E5        The Stock Tip        5  


In [ ]:
df.drop('title', axis=1, inplace=True)
print(df.head())

             airdate             rating  season season_episode  \
0   Wed, Oct 6, 1993  7.3\n/10\n (6.4K)       1          S1.E1   
1  Thu, May 31, 1990  7.5\n/10\n (5.5K)       1          S1.E2   
2   Thu, Jun 7, 1990    7.5\n/10\n (5K)       1          S1.E3   
3  Thu, Jun 14, 1990  7.3\n/10\n (4.9K)       1          S1.E4   
4  Thu, Jun 21, 1990  7.4\n/10\n (4.8K)       1          S1.E5   

         episode_title  episode  
0  Good News, Bad News        1  
1         The Stakeout        2  
2          The Robbery        3  
3       Male Unbonding        4  
4        The Stock Tip        5  


In [ ]:
# Convert 'airdate' to a datetime format
df['airdate'] = pd.to_datetime(df['airdate'], format='%a, %b %d, %Y', errors='coerce')


print(df['airdate'].head())


0   1993-10-06
1   1990-05-31
2   1990-06-07
3   1990-06-14
4   1990-06-21
Name: airdate, dtype: datetime64[ns]


In [ ]:
# Rearranging the order of the columns
new_order = ['season', 'season_episode', 'episode_title', 'airdate', 'rating']


df = df[new_order]

In [ ]:
print(df.head())

   season season_episode        episode_title    airdate             rating
0       1          S1.E1  Good News, Bad News 1993-10-06  7.3\n/10\n (6.4K)
1       1          S1.E2         The Stakeout 1990-05-31  7.5\n/10\n (5.5K)
2       1          S1.E3          The Robbery 1990-06-07    7.5\n/10\n (5K)
3       1          S1.E4       Male Unbonding 1990-06-14  7.3\n/10\n (4.9K)
4       1          S1.E5        The Stock Tip 1990-06-21  7.4\n/10\n (4.8K)


At the moment, the 'rating' column isn't particularly usable. I want to extract rating out of ten, split the columns into two, and turn the vote count into a numeric type.

In [ ]:
df['rating'] = df['rating'].astype(str)


df[['rating_value', 'votes_text']] = df['rating'].str.split('/10', expand=True)


df['votes_text'] = df['votes_text'].str.extract(r'\(([\d\.]+K?)\)', expand=False)

The 'votes_text' column now contains values such as '6.4K' etc. without parentheses, but I still need to turn the 'K' into numeric thousands using its scientific notation 1e-3.

In [ ]:

df['rating_value'] = df['rating_value'].astype(float)


def votes_num(votes_str):
    if pd.isna(votes_str):  
        return np.nan
    if votes_str.endswith('K'):
        return float(votes_str[:-1]) * 1e3
    return float(votes_str)  # If there's no 'K', it's just a straightforward number


df['total_votes'] = df['votes_text'].apply(votes_num)

# Drop the now-unnecessary 'votes_text' column
df.drop(columns=['votes_text'], inplace=True)

print(df[['season', 'season_episode', 'episode_title', 'airdate', 'rating_value', 'total_votes']].head())


   season season_episode        episode_title    airdate  rating_value  \
0       1          S1.E1  Good News, Bad News 1993-10-06           7.3   
1       1          S1.E2         The Stakeout 1990-05-31           7.5   
2       1          S1.E3          The Robbery 1990-06-07           7.5   
3       1          S1.E4       Male Unbonding 1990-06-14           7.3   
4       1          S1.E5        The Stock Tip 1990-06-21           7.4   

   total_votes  
0       6400.0  
1       5500.0  
2       5000.0  
3       4900.0  
4       4800.0  


In [ ]:
df.drop('rating', axis=1, inplace=True)

In [ ]:
df.head()

,season,season_episode,episode_title,airdate,rating_value,total_votes
0,1,S1.E1,"Good News, Bad News",1993-10-06,7.3,6400.0
1,1,S1.E2,The Stakeout,1990-05-31,7.5,5500.0
2,1,S1.E3,The Robbery,1990-06-07,7.5,5000.0
3,1,S1.E4,Male Unbonding,1990-06-14,7.3,4900.0
4,1,S1.E5,The Stock Tip,1990-06-21,7.4,4800.0


In [ ]:
df.to_csv('seinfeld_ratings_cleaned.csv', index=False)

I realised here that the dates for the first season hadn't been scraped properly for some reason, and with bigger fish to fry I decided to just edit them manually.

<h3> The dataset saga continues: extracting columns from a pre-existing dataset and concatenating them to my own </h3>

I found another dataset that included information about the writer(s) and director for each episode and wanted to include it in my own dataset to expand the scope of analysis.

Firstly, I need to isolate the two columns I needed from the dataset and write them to a new file:

In [ ]:
ep_info = pd.read_csv('/Users/sara/seinfeld/episode_info.csv') 

# Identifying the columns to isolate
wd_cols = ep_info[['writer', 'director']] 

wd_cols.to_csv('/Users/sara/seinfeld/csv/wd_columns.csv', index=False) 

In [ ]:
wd_cols.head()

,writer,director
0,"Larry David, Jerry Seinfeld",Art Wolff
1,"Larry David, Jerry Seinfeld",Tom Cherones
2,Matt Goldman,Tom Cherones
3,"Larry David, Jerry Seinfeld",Tom Cherones
4,"Larry David, Jerry Seinfeld",Tom Cherones


Concatenating the file containing the two extracted columns to my main dataset:

In [ ]:
import pandas as pd

seinfeld_df = pd.read_csv('/Users/sara/seinfeld/csv/ratings_cleanest.csv')
wd_colunns = pd.read_csv('/Users/sara/seinfeld/wd_colunns.csv')


ep_inf_tr = wd_colunns.iloc[:len(seinfeld_df)][['writer', 'director']]

comb_df = pd.concat([seinfeld_df, wd_colunns_tr], axis=1)

comb_df.to_csv('merged_seinfeld.csv', index=False)


In [ ]:
comb_df.head()

,season,season_episode,episode_title,airdate,rating_value,total_votes,writer,director
0,1,S1.E1,"Good News, Bad News",1989-07-05,7.3,6400.0,"Larry David, Jerry Seinfeld",Art Wolff
1,1,S1.E2,The Stakeout,1990-05-31,7.5,5500.0,"Larry David, Jerry Seinfeld",Tom Cherones
2,1,S1.E3,The Robbery,1990-06-07,7.5,5000.0,Matt Goldman,Tom Cherones
3,1,S1.E4,Male Unbonding,1990-06-14,7.3,4900.0,"Larry David, Jerry Seinfeld",Tom Cherones
4,1,S1.E5,The Stock Tip,1990-06-21,7.4,4800.0,"Larry David, Jerry Seinfeld",Tom Cherones


Next, I want to find out which writer-director pairings resulted in the highest episode ratings.
For this, grouping by writer and director to calculate the mean rating for the episodes they worked on together:

In [ ]:
wd_ratings = df.groupby(['writer', 'director'])['rating_value'].mean().sort_values(ascending=False).reset_index()

# Identifying the top 5 writer-director teams with the highest average ratings
wd_dream_teams = wd_ratings.head(5)

print(wd_dream_teams)


                                   writer            director  rating_value
0             Peter Mehlman, Carol Leifer        Tom Cherones           9.5
1         Peter Mehlman and Jill Franklyn       Andy Ackerman           9.0
2            Carol Leifer, Marjorie Gross  David Owen Trainor           8.9
3             David Mandel, Peter Mehlman       Andy Ackerman           8.9
4  Tom Gammill, Max Pross and Larry David       Andy Ackerman           8.9


Upon first glance it seems to have done the trick, but we're not accounting for the way the writers are grouped together in the file. In the 'writer' column, there is sometimes a comma between two individual names, or an 'and' joining them together.

Below we'll try again, this time accounting for the mistake.

In [ ]:
df['writer_list'] = df['writer'].str.replace(' and ', ', ').str.split(', ')

exploded_df = df.explode('writer_list')

ind_wd_ratings = exploded_df.groupby(['writer_list', 'director'])['rating_value'].mean().sort_values(ascending=False).reset_index()

wd_pairings = ind_wd_ratings.head(5)

print(ind_wd_ratings.head(5))


      writer_list            director  rating_value
0   Jill Franklyn       Andy Ackerman          9.00
1    Carol Leifer  David Owen Trainor          8.90
2    Carol Leifer        Tom Cherones          8.90
3  Marjorie Gross  David Owen Trainor          8.90
4     Larry Levin        Tom Cherones          8.75
